In [1]:
from langchain_groq import ChatGroq
from langchain_community.graphs import Neo4jGraph
from dotenv import load_dotenv
import os
import utils
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from langchain.text_splitter import CharacterTextSplitter
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain.schema import Document
load_dotenv()

True

In [2]:
graph=Neo4jGraph(
    url=os.environ.get("NEO4J_URI"),
    username=os.environ.get("NEO4J_USERNAME"),
    password=os.environ.get("NEO4J_PASSWORD")
)
graph

C:\Users\onlys\AppData\Local\Temp\ipykernel_29664\1306043822.py:1: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph=Neo4jGraph(


In [3]:
llm=ChatGroq(groq_api_key=os.environ.get("GROQ_API_KEY"),model_name="Llama-3.3-70b-Versatile")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000015B1CB64150>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000015B1CB65610>, model_name='Llama-3.3-70b-Versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
def process_doc(doc, chunk_size, chunk_overlap):
    splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = []
    for chunk in splitter.split_text(doc["text"]):
        chunks.append({"text": chunk, "source": doc["source"]})
    return chunks

def chunk_text(documents, chunk_size=500, chunk_overlap=50):
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(process_doc, documents, [chunk_size]*len(documents), [chunk_overlap]*len(documents)), total=len(documents)))

    return [chunk for result in results for chunk in result]

In [34]:
docx_folder = "../Document"

# Step 1: Extract Text from PDFs
print("Extracting text from PDFs in parallel...")
documents = utils.extract_text_from_docx_parallel(docx_folder)

cont=""

print(documents)
with open("documents.txt", "w") as f:
    f.write(documents[0]['text'])

print("Splitting text into chunks...")
chunks = chunk_text(documents)

print("Successfully split text into chunks.")
len(chunks)

Extracting text from PDFs in parallel...


100%|██████████| 1/1 [00:00<00:00,  2.40it/s]
Created a chunk of size 787, which is longer than the specified 500


[{'text': "The software development process\n\n\tThe development contract\n\nA decision to adopt and apply MISRA Guidelines should be taken at the outset of a project. The application of MISRA Guidelines will typically be one requirement among many of a contractual agreement between two parties, the organization commissioning the project (the acquirer) and the organization developing the code (the supplier). These parties may be different commercial entities or they may simply be different departments within the same organization. Both will need to be actively involved in the task of assuring compliance with MISRA Guidelines. MISRA Compliance is not a rigidly defined concept and acceptance criteria will be a matter for negotiation. However there are some clear principles which must be observed in order for the concept of compliance to have credibility.\n\n\n\n\tFramework\n\nMISRA Guidelines are intended to be used within the framework of a documented software development process. They 

Created a chunk of size 571, which is longer than the specified 500
  0%|          | 0/1 [00:00<?, ?it/s]Created a chunk of size 527, which is longer than the specified 500
Created a chunk of size 673, which is longer than the specified 500
Created a chunk of size 534, which is longer than the specified 500
Created a chunk of size 553, which is longer than the specified 500
Created a chunk of size 615, which is longer than the specified 500
Created a chunk of size 1105, which is longer than the specified 500
Created a chunk of size 521, which is longer than the specified 500
Created a chunk of size 743, which is longer than the specified 500
Created a chunk of size 769, which is longer than the specified 500
Created a chunk of size 515, which is longer than the specified 500
Created a chunk of size 750, which is longer than the specified 500
Created a chunk of size 652, which is longer than the specified 500
Created a chunk of size 692, which is longer than the specified 500
Created a 

Successfully split text into chunks.


169

In [6]:
documents = [
        Document(page_content=chunk["text"], metadata={"source": chunk["source"]})
        for chunk in chunks
]

In [7]:
llm_transformer = LLMGraphTransformer(llm=llm)

# Ensure documents are in the correct format
print("Formating documents...")
formatted_documents = [
	Document(page_content=doc.page_content, metadata=doc.metadata)
	for doc in documents
]
print("Successfully formatted documents.")

try:
	graph_documents = llm_transformer.convert_to_graph_documents(formatted_documents)
	print("Successfully converted documents to graph documents.")
except Exception as e:
	print(f"An error occurred: {e}")

Formating documents...
Successfully formatted documents.
Successfully converted documents to graph documents.


In [8]:
graph_documents[0].nodes

[Node(id='Software', type='Software development', properties={}),
 Node(id='Development_Contract', type='Contract', properties={})]

In [9]:
graph_documents[0].relationships

[Relationship(source=Node(id='Software', type='Software development', properties={}), target=Node(id='Development_Contract', type='Contract', properties={}), type='GOVERNED_BY', properties={})]

In [10]:
graph_documents

[GraphDocument(nodes=[Node(id='Software', type='Software development', properties={}), Node(id='Development_Contract', type='Contract', properties={})], relationships=[Relationship(source=Node(id='Software', type='Software development', properties={}), target=Node(id='Development_Contract', type='Contract', properties={}), type='GOVERNED_BY', properties={})], source=Document(metadata={'source': 'MISRA-Compliance-2020.docx'}, page_content='The software development process\n\n\tThe development contract')),
 GraphDocument(nodes=[Node(id='Misra Guidelines', type='Guidelines', properties={}), Node(id='Project', type='Project', properties={}), Node(id='Acquirer', type='Organization', properties={}), Node(id='Supplier', type='Organization', properties={})], relationships=[Relationship(source=Node(id='Misra Guidelines', type='Guidelines', properties={}), target=Node(id='Project', type='Project', properties={}), type='APPLIED_TO', properties={}), Relationship(source=Node(id='Acquirer', type='Or

In [11]:
graph.add_graph_documents(graph_documents)

[#DB60]  _: <CONNECTION> error: Failed to write data to connection ResolvedIPv4Address(('34.126.114.186', 7687)) (ResolvedIPv4Address(('34.126.114.186', 7687))): SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:2427)')
Unable to retrieve routing information
Transaction failed and will be retried in 1.1886464408000244s (Unable to retrieve routing information)


In [14]:
from langchain.chains.graph_qa.cypher import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(llm=llm,graph=graph,verbose=True,allow_dangerous_requests=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x0000015B67759B90>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000015B1CB64150>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000015B1CB65610>, model_name=

In [27]:
response=chain.invoke({"query":"What is Development_Contract"})

response



> Entering new GraphCypherQAChain chain...


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownLabelWarning} {category: UNRECOGNIZED} {title: The provided label is not in the database.} {description: One of the labels in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing label name is: Development_Contract)} {position: line: 1, column: 10, offset: 9} for query: 'MATCH (n:Development_Contract) RETURN n'


Generated Cypher:
MATCH (n:Development_Contract) RETURN n
Full Context:
[]

> Finished chain.


{'query': 'What is Development_Contract', 'result': "I don't know the answer."}